<a href="https://colab.research.google.com/github/Bryant1217/Climate--Forecast-Temperature/blob/main/climate_cleaning_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

#read other data

#GlobalLandTemperaturesByCity

In [ ]:
df= pd.read_csv('GlobalLandTemperaturesByCity.csv')
df.shape

(8599212, 7)

check the time range

In [ ]:
df['dt'].unique

<bound method Series.unique of 0          1743-11-01
1          1743-12-01
2          1744-01-01
3          1744-02-01
4          1744-03-01
              ...    
8599207    2013-05-01
8599208    2013-06-01
8599209    2013-07-01
8599210    2013-08-01
8599211    2013-09-01
Name: dt, Length: 8599212, dtype: object>

#Check Missing Data

In [ ]:
missing_values = df.isnull().sum()
print("Missing values in each column:")
print(missing_values)

Missing values in each column:
dt                                    0
AverageTemperature               364130
AverageTemperatureUncertainty    364130
City                                  0
Country                               0
Latitude                              0
Longitude                             0
dtype: int64


#Exclude year before 1900 (filtered_df)

In [ ]:
df['dt'] = pd.to_datetime(df['dt'])

# Filter the DataFrame for years between 1900 and 2013
filtered_df = df[(df['dt'].dt.year >= 1900) & (df['dt'].dt.year <= 2013)]
filtered_df= filtered_df.reset_index(drop=True)

# Print the filtered DataFrame
print(filtered_df)


                dt  AverageTemperature  AverageTemperatureUncertainty    City  \
0       1900-01-01              -0.989                          0.588   Århus   
1       1900-02-01              -2.799                          0.882   Århus   
2       1900-03-01               0.592                          0.429   Århus   
3       1900-04-01               4.630                          0.417   Århus   
4       1900-05-01               9.576                          0.521   Århus   
...            ...                 ...                            ...     ...   
4791145 2013-05-01              11.464                          0.236  Zwolle   
4791146 2013-06-01              15.043                          0.261  Zwolle   
4791147 2013-07-01              18.775                          0.193  Zwolle   
4791148 2013-08-01              18.025                          0.298  Zwolle   
4791149 2013-09-01                 NaN                            NaN  Zwolle   

             Country Latitu

#check missing values after year 1900

In [ ]:
missing_values = filtered_df.isnull().sum()
print("Missing values in each column:")
print(missing_values)

Missing values in each column:
dt                                  0
AverageTemperature               3070
AverageTemperatureUncertainty    3070
City                                0
Country                             0
Latitude                            0
Longitude                           0
dtype: int64


In [ ]:
# Check for missing values
missing_values = filtered_df.isnull()

# Reset the index before applying the groupby function
missing_values_reset_index = missing_values.reset_index(drop=True)

# Calculate consecutive occurrences of missing values
missing_values_reset_index['consecutive_count'] = (
    missing_values_reset_index.groupby((~missing_values_reset_index).cumsum().iloc[:, 0])
    .cumcount() + 1
)

# Determine if missing values are continuous or not
continuous_missing = missing_values_reset_index.groupby('consecutive_count').size().idxmax()
if continuous_missing == 1:
    print("Missing values are continuous.")
else:
    print("Missing values are not continuous.")

Missing values are continuous.


Check missing values max_continuous_missing=5

In [ ]:
def check_missing_sequence(filtered_df, max_continuous_missing=5):
    missing_count = 0
    for value in filtered_df:
        if pd.isnull(value):
            missing_count += 1
            if missing_count > max_continuous_missing:
                return False
        else:
            missing_count = 0
    return True
print(check_missing_sequence(filtered_df))

True


#Missing value(1)

check mising value for each country

In [ ]:
print(len(filtered_df['Country'].unique()))
# Group the DataFrame by 'Country'
country_groups = filtered_df.groupby('Country')

# Loop through each group and print the count of missing values in 'AverageTemperature'
for country, group_df in country_groups:
    missing_values_count = group_df['AverageTemperature'].isna().sum()
    print(f"Country: {country}, Missing Values Count: {missing_values_count}")


159
Country: Afghanistan, Missing Values Count: 8
Country: Albania, Missing Values Count: 3
Country: Algeria, Missing Values Count: 5
Country: Angola, Missing Values Count: 6
Country: Argentina, Missing Values Count: 32
Country: Armenia, Missing Values Count: 3
Country: Australia, Missing Values Count: 14
Country: Austria, Missing Values Count: 5
Country: Azerbaijan, Missing Values Count: 1
Country: Bahamas, Missing Values Count: 0
Country: Bahrain, Missing Values Count: 1
Country: Bangladesh, Missing Values Count: 19
Country: Belarus, Missing Values Count: 8
Country: Belgium, Missing Values Count: 7
Country: Benin, Missing Values Count: 6
Country: Bolivia, Missing Values Count: 6
Country: Bosnia And Herzegovina, Missing Values Count: 5
Country: Botswana, Missing Values Count: 1
Country: Brazil, Missing Values Count: 220
Country: Bulgaria, Missing Values Count: 7
Country: Burkina Faso, Missing Values Count: 2
Country: Burma, Missing Values Count: 20
Country: Burundi, Missing Values Cou

make sure if the missing value is not the first data of each country

In [ ]:
for country, group_df in country_groups:
    # Check if there are any missing values in the 'AverageTemperature' column
    if group_df['AverageTemperature'].isna().any():
        # Get the index of the last row for the current country
        last_index = group_df.index[0]

        # Get the index of the row containing the missing value
        missing_index = group_df[group_df['AverageTemperature'].isna()].index[0]

        # Check if the missing index is equal to the last index
        if last_index == missing_index:
            print(f"Country: {country}, Missing value is the first data point.")
        else:
            print(f"Country: {country}, Not the first data point.")
    else:
        print(f"Country: {country}, No missing values in 'AverageTemperature' column.")


Country: Afghanistan, Not the first data point.
Country: Albania, Not the first data point.
Country: Algeria, Not the first data point.
Country: Angola, Not the first data point.
Country: Argentina, Not the first data point.
Country: Armenia, Not the first data point.
Country: Australia, Not the first data point.
Country: Austria, Not the first data point.
Country: Azerbaijan, Not the first data point.
Country: Bahamas, No missing values in 'AverageTemperature' column.
Country: Bahrain, Not the first data point.
Country: Bangladesh, Not the first data point.
Country: Belarus, Not the first data point.
Country: Belgium, Not the first data point.
Country: Benin, Not the first data point.
Country: Bolivia, Not the first data point.
Country: Bosnia And Herzegovina, Not the first data point.
Country: Botswana, Not the first data point.
Country: Brazil, Not the first data point.
Country: Bulgaria, Not the first data point.
Country: Burkina Faso, Not the first data point.
Country: Burma, Not 

Dealing with missing value (replace by previous valid value)

In [ ]:
# Fill NaN values in 'AverageTemperature' column with values from the previous non-NaN value
df1=filtered_df.copy()
df1['AverageTemperature'] = df1['AverageTemperature'].fillna(method='ffill')
df1['AverageTemperatureUncertainty'] = df1['AverageTemperatureUncertainty'].fillna(method='ffill')
df1.head(50)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1900-01-01,-0.989,0.588,Århus,Denmark,57.05N,10.33E
1,1900-02-01,-2.799,0.882,Århus,Denmark,57.05N,10.33E
2,1900-03-01,0.592,0.429,Århus,Denmark,57.05N,10.33E
3,1900-04-01,4.630,0.417,Århus,Denmark,57.05N,10.33E
4,1900-05-01,9.576,0.521,Århus,Denmark,57.05N,10.33E
5,1900-06-01,15.888,0.592,Århus,Denmark,57.05N,10.33E
6,1900-07-01,16.988,0.706,Århus,Denmark,57.05N,10.33E
7,1900-08-01,16.517,0.385,Århus,Denmark,57.05N,10.33E
8,1900-09-01,12.984,0.481,Århus,Denmark,57.05N,10.33E
9,1900-10-01,8.041,0.453,Århus,Denmark,57.05N,10.33E


In [ ]:
# Filter the DataFrame for Denmark
denmark_data = df1[df1['Country'] == 'Denmark']
A=denmark_data['City']== 'Århus'
A1=denmark_data[A]
# Create a line chart using Plotly Express
figA1 = px.line(A1, x='dt', y='AverageTemperature', title='Average Temperature in Denmark,Århus(missing values replace by previous valid data)')
figA1.update_xaxes(title='Date', tickangle=45)
figA1.update_yaxes(title='Average Temperature')

# Show the plot
figA1.show()

masha’s comments —> look at missing value treatment again, is it random missing or not, look at cllustering countries based on paramters you have and exploree if there is any correlation in climate change and geography, other sociol economic factors

#Missing value(2)


In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

df2=filtered_df.copy()

train = df2['AverageTemperature'].dropna()  # Remove rows with missing values for training
test = df2[filtered_df['AverageTemperature'].isna()]  # Rows with missing values for testing

# Fit winter model
model = ExponentialSmoothing(train, seasonal='add', seasonal_periods=12).fit()
imputed_values = model.predict(start=test.index.min(), end=test.index.max())

# Replace missing values with imputed values
df2.loc[test.index, 'AverageTemperature'] = imputed_values


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [ ]:
missing_value = df2.isnull().sum()
print("Missing values in each column:")
print(missing_value)

Missing values in each column:
dt                                  0
AverageTemperature                  0
AverageTemperatureUncertainty    3070
City                                0
Country                             0
Latitude                            0
Longitude                           0
dtype: int64


In [ ]:
#df2.drop(columns=['AverageTemperatureUncertainty'], inplace=True)
df2.tail()

,dt,AverageTemperature,City,Country,Latitude,Longitude
4791145,2013-05-01,11.464000,Zwolle,Netherlands,52.24N,5.26E
4791146,2013-06-01,15.043000,Zwolle,Netherlands,52.24N,5.26E
4791147,2013-07-01,18.775000,Zwolle,Netherlands,52.24N,5.26E
4791148,2013-08-01,18.025000,Zwolle,Netherlands,52.24N,5.26E
4791149,2013-09-01,15.160778,Zwolle,Netherlands,52.24N,5.26E


In [ ]:
df2.to_csv('1900-2013.csv', index=False)

#plot

In [ ]:
import plotly.express as px

# Filter the DataFrame for Denmark
denmark_data = filtered_df[filtered_df['Country'] == 'Denmark']

# Create a line chart using Plotly Express
fig = px.line(denmark_data, x='dt', y='AverageTemperature', title='Average Temperature in Denmark')
fig.update_xaxes(title='Date', tickangle=45)
fig.update_yaxes(title='Average Temperature')

# Show the plot
fig.show()

In [ ]:
# Filter the DataFrame for Denmark
denmark_data2 = df2[df2['Country'] == 'Denmark']
B=denmark_data2['City']== 'Århus'
A2=denmark_data[B]
# Create a line chart using Plotly Express
figA2 = px.line(A2, x='dt', y='AverageTemperature', title='Average Temperature in Denmark, Århus(missing values replace by winter model)')
figA2.update_xaxes(title='Date', tickangle=45)
figA2.update_yaxes(title='Average Temperature')

# Show the plot
figA2.show()

#compare 1

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create a subplot figure
fig = make_subplots(rows=1, cols=2, subplot_titles=('(missing values replace by previous valid data)', '(missing values replace by winter model)'))

# Add traces from the first Plotly Express figure to the subplot
for trace in figA1.data:
   fig.add_trace(trace, row=1, col=1)

# Add traces from the second Plotly Express figure to the subplot
for trace in figA2.data:
    fig.add_trace(trace, row=1, col=2)

# Update layout if needed
fig.update_layout(height=600, width=1700, title_text="A1 vs A2")
fig.show()

In [ ]:
A1.describe()

,AverageTemperature,AverageTemperatureUncertainty
count,1365.000000,1365.000000
mean,8.027705,0.333241
std,6.383653,0.126397
min,-7.816000,0.077000
25%,2.562000,0.247000
50%,7.404000,0.311000
75%,13.920000,0.396000
max,20.883000,0.956000


In [ ]:
A2.describe()

,AverageTemperature,AverageTemperatureUncertainty
count,1365.000000,1364.000000
mean,8.024677,0.333186
std,6.380172,0.126427
min,-7.816000,0.077000
25%,2.562000,0.247000
50%,7.404000,0.311000
75%,13.889000,0.396000
max,20.883000,0.956000


#compare 2

In [ ]:
JP_data = df1[df1['Country'] == 'Japan']
JP_data

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
24570,1900-01-01,-0.570,0.430,Abiko,Japan,36.17N,139.23E
24571,1900-02-01,0.932,0.528,Abiko,Japan,36.17N,139.23E
24572,1900-03-01,4.175,0.214,Abiko,Japan,36.17N,139.23E
24573,1900-04-01,10.210,0.342,Abiko,Japan,36.17N,139.23E
24574,1900-05-01,16.124,0.469,Abiko,Japan,36.17N,139.23E
...,...,...,...,...,...,...,...
4724260,2013-05-01,16.757,0.396,Zama,Japan,36.17N,139.23E
4724261,2013-06-01,20.927,0.489,Zama,Japan,36.17N,139.23E
4724262,2013-07-01,24.645,0.404,Zama,Japan,36.17N,139.23E
4724263,2013-08-01,26.364,0.461,Zama,Japan,36.17N,139.23E


In [ ]:
# Filter the DataFrame for Japan (missing values replace by previous valid data
JP_data = df1[df1['Country'] == 'Japan']
Z=JP_data['City']== 'Zama'
Z1=JP_data[Z]
# Create a line chart using Plotly Express
figZ1 = px.line(Z1, x='dt', y='AverageTemperature', title='Average Temperature in Japan, Zama(missing values replace by previous valid data)')
figZ1.update_xaxes(title='Date', tickangle=45)
figZ1.update_yaxes(title='Average Temperature')

# Show the plot
figZ1.show()

In [ ]:
# Filter the DataFrame for Japan (missing values replace by winter model
JP_data = df2[df2['Country'] == 'Japan']
ZZ=JP_data['City']== 'Zama'
Z2=JP_data[ZZ]
# Create a line chart using Plotly Express
figZ2 = px.line(Z2, x='dt', y='AverageTemperature', title='Average Temperature in Japan, Zama(missing values replace by winter model)')
figZ2.update_xaxes(title='Date', tickangle=45)
figZ2.update_yaxes(title='Average Temperature')

# Show the plot
figZ2.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create a subplot figure
fig = make_subplots(rows=1, cols=2, subplot_titles=('(missing values replace by previous valid data)', '(missing values replace by winter model)'))

# Add traces from the first Plotly Express figure to the subplot
for trace in figZ1.data:
   fig.add_trace(trace, row=1, col=1)

# Add traces from the second Plotly Express figure to the subplot
for trace in figZ2.data:
    fig.add_trace(trace, row=1, col=2)

# Update layout if needed
fig.update_layout(height=600, width=1700, title_text="Z1 vs Z2")
fig.show()

In [ ]:
Z1.describe()

,AverageTemperature,AverageTemperatureUncertainty
count,1365.000000,1365.000000
mean,12.745151,0.264748
std,8.192528,0.104056
min,-1.580000,0.049000
25%,4.718000,0.195000
50%,13.205000,0.250000
75%,20.313000,0.313000
max,27.295000,1.120000


In [ ]:
Z2.describe()

,AverageTemperature,AverageTemperatureUncertainty
count,1364.000000,1364.000000
mean,12.735166,0.264604
std,8.187220,0.103958
min,-1.580000,0.049000
25%,4.716500,0.195000
50%,13.175000,0.250000
75%,20.310750,0.313000
max,27.295000,1.120000


#missing value for whole dataset


In [ ]:
df3=df.copy()

train = df3['AverageTemperature'].dropna()  # Remove rows with missing values for training
test = df3[df3['AverageTemperature'].isna()]  # Rows with missing values for testing

# Fit winter model
model = ExponentialSmoothing(train, seasonal='add', seasonal_periods=12).fit()
imputed_values = model.predict(start=test.index.min(), end=test.index.max())

# Replace missing values with imputed values
df3.loc[test.index, 'AverageTemperature'] = imputed_values

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [ ]:
#df3.drop(columns=['AverageTemperatureUncertainty'], inplace=True)
df3.to_csv('data.csv', index=False)

In [ ]:
df4= pd.read_csv('data.csv')
missing_values = df4.isnull().sum()
print("Missing values in each column:")
print(missing_values)

Missing values in each column:
dt                    0
AverageTemperature    0
City                  0
Country               0
Latitude              0
Longitude             0
dtype: int64
